In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras import layers,models
from PIL import Image
# import torch
# import torchvision.transforms as T

In [2]:
IMAGE_SIZE=(256,256)
dataset=tf.keras.preprocessing.image_dataset_from_directory('C:\\Users\\ALVI COMPUTER\\Desktop\\Data_potatot_disease',
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE,
                                                            batch_size=32)

Found 2152 files belonging to 3 classes.


In [3]:
dataset.class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [4]:
len(dataset)

68

In [5]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
image_label=['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [17]:
for image_batch,label in dataset.take(1):
    print(label.numpy())
    print(image_batch.shape)
    image_np = image_batch[0]
    img_path = image_np 
    img = Image.open(img_path)
    img.show()
    
    

[1 1 0 1 0 1 0 2 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 0 2 1 0]
(32, 256, 256, 3)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'read'

In [4]:
def partition(data,shuffle=True,shuffle_size=10000):
    if shuffle:
        data=data.shuffle(shuffle_size,seed=12)
        train_size=0.8*len(data)
        train_data=data.take(int(train_size))
        rem_data=data.skip(int(train_size))
        test_size=len(data)*0.1
        test_data=rem_data.take(int(test_size))
        val_data=rem_data.skip(int(test_size))
        return train_data,test_data,val_data
        

In [5]:
train_data,test_data,val_data=partition(dataset)

In [6]:
len(train_data)

54

In [7]:
len(val_data)

8

In [8]:
len(test_data)

6

In [9]:
train_data=train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data=test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_data=val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [10]:
resize_rescale_layer=tf.keras.Sequential(layers.experimental.preprocessing.Resizing(256,256),
                                          layers.experimental.preprocessing.Rescaling(1.0/255))

In [11]:
data_augmentation_layer=tf.keras.Sequential(layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                            layers.experimental.preprocessing.RandomRotation(0.2))

In [12]:
channels=3
batch_size=32
image_size=256


In [13]:
input_shape=(image_size,image_size,channels)
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255),
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

                        

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [15]:
model.fit(train_data,
          epochs=30,
          batch_size=32,
          verbose=1,
          validation_data=val_data)

Epoch 1/30
54/54 [==============================] - 117s 2s/step - loss: 0.6550 - accuracy: 0.7135 - val_loss: 0.3757 - val_accuracy: 0.8750
Epoch 2/30
54/54 [==============================] - 100s 2s/step - loss: 0.2126 - accuracy: 0.9144 - val_loss: 0.2582 - val_accuracy: 0.8750
Epoch 3/30
54/54 [==============================] - 92s 2s/step - loss: 0.2657 - accuracy: 0.9010 - val_loss: 0.1693 - val_accuracy: 0.9336
Epoch 4/30
54/54 [==============================] - 91s 2s/step - loss: 0.1521 - accuracy: 0.9427 - val_loss: 0.0951 - val_accuracy: 0.9727
Epoch 5/30
54/54 [==============================] - 90s 2s/step - loss: 0.1284 - accuracy: 0.9520 - val_loss: 0.2793 - val_accuracy: 0.8633
Epoch 6/30
54/54 [==============================] - 91s 2s/step - loss: 0.1319 - accuracy: 0.9514 - val_loss: 0.5199 - val_accuracy: 0.7773
Epoch 7/30
54/54 [==============================] - 90s 2s/step - loss: 0.1161 - accuracy: 0.9583 - val_loss: 0.4332 - val_accuracy: 0.8398
Epoch 8/30
54/54 [

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 256, 256, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 256, 256, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 256, 256, 3)      0         
 tation)                                                         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                      

In [18]:
res=model.evaluate(test_data)

6/6 [==============================] - 4s 331ms/step - loss: 0.3149 - accuracy: 0.8958


In [20]:
model_version=1
model.save(f"C:\\Users\\ALVI COMPUTER\\Desktop\\CNN_Potato_Disease\\{model_version}")

INFO:tensorflow:Assets written to: C:\Users\ALVI COMPUTER\Desktop\CNN_Potato_Disease\1\assets


INFO:tensorflow:Assets written to: C:\Users\ALVI COMPUTER\Desktop\CNN_Potato_Disease\1\assets


In [21]:
new_model=tf.keras.models.load_model('1')

In [22]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_1 (Resizing)       (None, 256, 256, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 random_flip_1 (RandomFlip)  (None, 256, 256, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 256, 256, 3)      0         
 tation)                                                         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                      

In [23]:
new_model.evaluate(test_data)

6/6 [==============================] - 2s 296ms/step - loss: 0.3149 - accuracy: 0.8958


[0.31493842601776123, 0.8958333134651184]